# Plan
- [x] Load all reviews into a data structure in memory
- [x] Apply tokenization to each review (base step)
- [ ] Choose and apply three pathways through the feature generation process
    - _Remember that you will need to justify your choices with reference to the accuracy that is achieved with each feature set_

A simple idea for a datastructure that holds the dataset in memory is:
```Python
dataset = [
    {
        "id": 1,
        "rating": 7,
        "polarity": 1,
        "contents:": "Lorem Ipsum"
    },
    ...
]
```

Later, this should be a `pandas` dataframe.

## Imports & Constants

In [53]:
import os
import copy
import math
import string
import numpy as np
import pandas as pd
import numpy.typing as npt
from typing_extensions import Self
from typing import TypeAlias, Tuple
from functools import cached_property
from collections import Counter, defaultdict
from sklearn.model_selection import StratifiedShuffleSplit

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer
from nltk.util import ngrams, everygrams

POSITIVE_REVIEWS_DIR = "./data/pos/"
NEGATIVE_REVIEWS_DIR = "./data/neg/"

[nltk_data] Downloading package punkt to /home/sowell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/sowell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sowell/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sowell/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Dataset Types

In [25]:
class TextualDataPoint:

    def __init__(self, file_path: str):
        self.file_path = file_path
        self.contents = self._extract_contents()

    @cached_property
    def basename(self) -> str:
        return os.path.basename(self.file_path)

    @cached_property
    def filename(self) -> str:
        return os.path.splitext(self.basename)[0]
    
    def _extract_contents(self) -> str:
        with open(self.file_path, "r") as file:
            return file.read()
        
    def as_dict(self) -> dict:
        return {
            'contents': self.contents,
        }


class Review(TextualDataPoint):

    def __init__(self, file_path: str):
        super().__init__(file_path)
        self.id = self._parse_id()
        self.rating = self._parse_rating()
        self.polarity = self._determine_polarity()

    def _parse_id(self) -> int:
        return int(self.filename.split("_")[0])

    def _parse_rating(self) -> int:
        return int(self.filename.split("_")[1])
    
    def _determine_polarity(self) -> int:
        if self.rating is None:
            self.rating = self._parse_rating()

        if self.rating <= 4:
            return 0
        elif self.rating >= 7:
            return 1
        else:
            raise ValueError(f"unexpected rating: {self.rating}")

    def as_dict(self) -> dict:
        return {
            'id': self.id,
            'rating': self.rating,
            'polarity': self.polarity,
        } | super().as_dict()

In [36]:
class IterableSet:

    datapoint_class: TypeAlias = TextualDataPoint

    def __init__(self, datapoints: list[datapoint_class]):
        self.datapoints = datapoints

        # To keep track of the current iteration position.
        self.index = 0

    def first(self) -> datapoint_class:
        return self.datapoints[0]

    def last(self) -> datapoint_class:
        return self.datapoints[-1]
    
    def __len__(self) -> int:
        return len(self.datapoints)
    
    def __iter__(self) -> Self:
        # Reset the index whenever starting a new iteration.
        self.index = 0
        return self
        
    def __next__(self) -> datapoint_class:
        # Make sure there are more datapoints to yield.
        if self.index < len(self.datapoints):
            result = self.datapoints[self.index]
            self.index += 1
            return result
        else:
            # No more datapoints -> raise StopIteration exception.
            raise StopIteration

    def as_lower_representation(self) -> list[dict]:
        return [
            datapoint.as_dict()
            for datapoint in self.datapoints
        ]
    
    def as_df(self) -> pd.DataFrame:
        return pd.DataFrame(self.as_lower_representation())

    def to_csv(self, file_path: str) -> None:
        return self.as_df().to_csv(file_path)
    

class SplitableSet(IterableSet):

    def to_csv_as_train_dev_test_sets(
            self, 
            output_dir: str, 
            target_variable_name: str, 
            dev_test_size: float = 0.3, 
            random_state: int = 42
        ) -> None:
        train, dev, test = self.as_train_dev_test_dfs(
            target_variable_name, dev_test_size, random_state)
        
        if not os.path.exists(output_dir):
            os.mkdir(output_dir)

        train.to_csv(output_dir+"train.csv")
        dev.to_csv(output_dir+"dev.csv")
        test.to_csv(output_dir+"test.csv")

    def as_training_dataframe(self, target_variable_name: str) -> pd.DataFrame:
        salient_columns = ["contents", target_variable_name]
        column_rename_map = {"contents": "X", target_variable_name: "y"}

        training_dataframe = self.as_df()[salient_columns]
        training_dataframe.rename(columns=column_rename_map, inplace=True)

        return training_dataframe
    
    def as_train_dev_test_arrays(
            self, 
            target_variable_name: str, 
            dev_test_size: float = 0.3, 
            random_state: int = 42
        ) -> Tuple[npt.NDArray, npt.NDArray, npt.NDArray]:
        train, dev, test = self.as_train_dev_test_dfs(
            target_variable_name,
            dev_test_size,
            random_state
        )

        return (
            self._convert_to_nd_array(train.X),
            train.y.values,
            self._convert_to_nd_array(dev.X),
            dev.y.values,
            self._convert_to_nd_array(test.X),
            test.y.values,
        )

    def as_train_dev_test_dfs(
            self, 
            target_variable_name: str, 
            dev_test_size: float = 0.3, 
            random_state: int = 42
        ) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
        X_y_entire = self.as_training_dataframe(target_variable_name)

        # Split the data into train and dev+test sets in a ratio of:
        #  -> (1-dev_test_size):(dev_test_size)
        initial_splitter = StratifiedShuffleSplit(
            n_splits=1, test_size=dev_test_size, random_state=random_state)
        train_indexes, test_indexes = next(
            initial_splitter.split(X_y_entire.X, X_y_entire.y))

        X_y_train = X_y_entire.iloc[train_indexes]
        X_y_test_dev = X_y_entire.iloc[test_indexes]

        # Split the dev + test set into dev and test sets in a 50:50 ratio.
        final_splitter = StratifiedShuffleSplit(
            n_splits=1, test_size=0.5, random_state=random_state)
        dev_indexes, test_indexes = next(
            final_splitter.split(X_y_test_dev.X, X_y_test_dev.y))

        X_y_dev = X_y_test_dev.iloc[dev_indexes]
        X_y_test = X_y_test_dev.iloc[test_indexes]

        return X_y_train, X_y_dev, X_y_test

    def _convert_to_nd_array(self, series: pd.Series) -> npt.NDArray:
        return np.vstack(series.apply(np.array))


class DataSet(SplitableSet):

    def __init__(self, dirs: list[str]):
        super().__init__(None)
        self.dirs = dirs
        
    def load(self) -> Self:
        self.datapoints = [
            self.datapoint_class(directory + file)
            for directory in self.dirs
            for file in os.listdir(directory)
        ]

        return self
    
    def as_lower_representation(self) -> list[dict]:
        # Ensure the dataset has been loaded.
        if self.datapoints is None:
            self.load()

        return super().as_lower_representation()

    def __iter__(self) -> Self:
        # Ensure the dataset has been loaded.
        if self.datapoints is None:
            self.load()

        return super().__iter__()
    

class ReviewDataSet(DataSet):

    datapoint_class: TypeAlias = Review

## Preprocessing

In [37]:
class Preprocessor:

    def __init__(self, dataset: DataSet):
        # We don't want to modify the original dataset.
        self.dataset = copy.deepcopy(dataset)
        # Tokenization is the first preprocessing step of most NLP applications.
        self.tokenize()

    def tokenize(self) -> Self:
        for datapoint in self.dataset:
            if isinstance(datapoint.contents, list):
                # This datapoint has already been tokenized.
                continue

            datapoint.contents = nltk.word_tokenize(datapoint.contents)
        
        return self

## Feature Set Generation

[Stemming (NTLK)](https://www.nltk.org/howto/stem.html)  
[A comparison of Stemming Algorithms & Lemmatization Algorithms](https://stackoverflow.com/questions/24647400/what-is-the-best-stemming-method-in-python)
- `PorterStemmer` is apparently one of the most aggresive `nltk` stemmers
    - It appears the choice of stemmer has a significant impact on performance
- `SnowballStemmer` appears to be a lighter middle-ground
- Lemmatizers are usually "lighter" than stemmers, but they cannot handle unknown words

In [38]:
class FeatureSet(SplitableSet):

    def __init__(self, dataset: DataSet):
        super().__init__(dataset.datapoints)

    def compare_with(self, other_set: Self):
        set1_dp1 = self.first().contents
        set2_dp1 = other_set.first().contents
        max_length_set1 = len(max(set1_dp1, key=len))

        print("Comparing feature sets Self and Other:")
        for token1, token2 in zip(set1_dp1, set2_dp1):
            empty_space = " " * (max_length_set1 - len(token1))
            print(f"Set A: {token1} {empty_space}| Set B: {token2}")

In [39]:
class FeatureSetGenerator(Preprocessor):

    def create_n_grams(self, n: int) -> FeatureSet:
        for datapoint in self.dataset:
            datapoint.contents = list(ngrams(datapoint.contents, n))

        return FeatureSet(self.dataset)
    
    def create_everygrams(self, max_n: int) -> FeatureSet:
        for datapoint in self.dataset:
            datapoint.contents = list(everygrams(datapoint.contents, max_len=max_n))

        return FeatureSet(self.dataset)
    
    def to_lowercase(self) -> Self:
        for datapoint in self.dataset:
            datapoint.contents = [token.lower() for token in datapoint.contents]

        return self
    
    def remove_stopwords(self) -> Self:
        distinct_stopwords = set(stopwords.words('english'))

        for datapoint in self.dataset:
            datapoint.contents = [token for token in datapoint.contents if token not in distinct_stopwords]

        return self
    
    def remove_punctuation(self) -> Self: 
        for datapoint in self.dataset:
            datapoint.contents = [token for token in datapoint.contents if token not in string.punctuation]

        return self

    def lemmatize(self) -> Self:
        lmtzr = WordNetLemmatizer()

        for datapoint in self.dataset:
            datapoint.contents = [lmtzr.lemmatize(token) for token in datapoint.contents]

        return self

    def stem(self) -> Self:
        # Making the assumption that all datapoints are in English.
        stmr = SnowballStemmer("english")

        for datapoint in self.dataset:
            datapoint.contents = [stmr.stem(token) for token in datapoint.contents]

        return self

## Feature Set Normalisation

- This needs to be much faster.
- Are we actually calcuating word frequencies or just word occurences (are these different?)

In [40]:
class FeatureSetNormalizer:

    def __init__(self, feature_set: FeatureSet):
        # We don't want to modify the original feature set.
        self.feature_set = copy.deepcopy(feature_set)

        self.normalized = False
        self.shared_vocabulary = self._collect_shared_vocabulary()

    @cached_property
    def num_datapoints(self) -> int:
        return len(self.feature_set)

    def perform_tf(self) -> FeatureSet:
        self._calculate_term_frequencies()

        for datapoint in self.feature_set:
            datapoint.contents = [
                datapoint.term_frequencies.get(token, 0)
                for token in self.shared_vocabulary
            ]

        return self.feature_set

    def perform_tf_idf(self) -> FeatureSet:
        self._calculate_term_frequencies()
        self.idfs = self._calculate_idfs()

        for datapoint in self.feature_set:
            datapoint.contents = [
                (datapoint.term_frequencies.get(token, 0) * self.idfs.get(token))
                for token in self.shared_vocabulary
            ]

        return self.feature_set

    def peform_ppmi(self) -> FeatureSet:
        raise NotImplementedError

    def _collect_shared_vocabulary(self) -> set:
        return set(sorted({
            token
            for datapoint in self.feature_set
            for token in datapoint.contents
        }))
    
    def _calculate_idfs(self) -> dict:
        # We calculate the document frequencies by creating a unique set of tokens for
        # each datapoint (i.e., for each document in the set, counting each token once
        # per document regardless of its frequency within the document itself). The Counter
        # then aggregates these sets across all datapoints, counting the number of documents
        # in which each distinct token appears. This gives us the document frequency for
        # each term in the `shared_vocabulary`.
        document_frequencies = Counter(
            token
            for datapoint in self.feature_set
            for token in set(datapoint.contents)
        )

        return {
            token: math.log(self.num_datapoints / (doc_frequency + 1))
            for token, doc_frequency in document_frequencies.items()
        }
    
    def _calculate_term_frequencies(self) -> None:
        for datapoint in self.feature_set:
            num_tokens = len(datapoint.contents)
            term_occurences = Counter(datapoint.contents)
            
            # Normalise the term occurences by dividing them by the length of the datapoint.
            datapoint.term_frequencies = {
                token: count / num_tokens
                for token, count in term_occurences.items()
            }

    def perform_fast_tf_idf(self) -> FeatureSet:
        vocab_to_index = {word: idx for idx, word in enumerate(self.shared_vocabulary)}
        num_documents = len(self.feature_set)
        num_vocab = len(self.shared_vocabulary)
        
        # TF matrix where each row corresponds to a document, and each column corresponds to a term.
        tf_matrix = np.zeros(shape=(num_documents, num_vocab), dtype=float)
        
        # Document frequency (DF) counter for counting in how many documents a term appears.
        df_counter = defaultdict(int)
        
        # Populate TF matrix and DF counter
        for doc_idx, datapoint in enumerate(self.feature_set):
            # Count term occurrences in the document
            term_occurrences = Counter(datapoint.contents)
            for term, count in term_occurrences.items():
                if term in vocab_to_index:
                    index = vocab_to_index[term]
                    tf_matrix[doc_idx, index] = count  # Raw count for TF (to be normalized later)
                    df_counter[term] += 1
        
        # Normalize TF matrix row-wise (divide by the number of terms in each document)
        doc_lengths = np.array([len(dp.contents) for dp in self.feature_set])
        tf_matrix = tf_matrix / doc_lengths[:, None]  # Broadcasting division
        
        # Convert the DF counter into an array of IDF values
        # idf_array = np.log((1 + num_documents) / (1 + np.array([df_counter[term] for term in self.shared_vocabulary]))) + 1
        idf_array = np.log((num_documents) / (1 + np.array([df_counter[term] for term in self.shared_vocabulary])))
        
        # TF-IDF calculation by multiplying the TF matrix by the IDF values
        # The transpose on idf_array is necessary for broadcasting to correct dimension
        tfidf_matrix = tf_matrix * idf_array
        
        # Update datapoint contents with tf-idf values
        for doc_idx, datapoint in enumerate(self.feature_set):
            datapoint.contents = tfidf_matrix[doc_idx, :].tolist()

        return self.feature_set

## Data

In [41]:
dataset = ReviewDataSet([POSITIVE_REVIEWS_DIR, NEGATIVE_REVIEWS_DIR]).load()

In [42]:
feature_set_b = FeatureSetGenerator(dataset)\
    .remove_punctuation()\
    .lemmatize()\
    .create_n_grams(1)

feature_set_c = FeatureSetGenerator(dataset)\
    .remove_stopwords()\
    .remove_punctuation()\
    .lemmatize()\
    .create_n_grams(1)

In [ ]:
feature_set_b.compare_with(feature_set_c)

In [43]:
normalizer = FeatureSetNormalizer(feature_set_c)

In [44]:
normalized_feature_set = normalizer.perform_fast_tf_idf()

## Train, Dev, Test Splits

In [45]:
X_train, y_train, X_dev, y_dev, X_test, y_test = normalized_feature_set.as_train_dev_test_arrays("polarity", 0.3)

In [49]:
train, dev, test = normalized_feature_set.as_train_dev_test_dfs("polarity", 0.3)

In [51]:
dev.head()

,X,y
347,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
668,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
2391,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
1125,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
3744,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0


## Save Original Dataset as CSV File

In [52]:
dataset.to_csv_as_train_dev_test_sets("./data/bert/", "polarity", 0.3)